<a href="https://colab.research.google.com/github/obscraft23/ipy-mcbe-edit/blob/master/mcberepair_wrapper_jp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

これは[mcberepair](https://github.com/reedacartwright/mcberepair)のjupyter notebookラッパーとして、試験的に作成したものです. <br>
ワールド負荷の原因となるpending tickデータやリモートプレーヤーデータの削除ができます.

In [ ]:
#@title <p> &darr; 開始するには再生ボタン &#9654;&#65039; を1回押してください. 警告が出る場合は「このまま実行する」を選択してください.<p> { vertical-output: true }

from IPython.utils import io
import os
import subprocess
import tqdm.notebook
from google.colab import files
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import datetime

TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt}'
print('\033[1m'+"[初期設定]"+'\033[0m'+" しばらくお待ちください(1分ほどかかる場合があります).")
# initial settings
try:
  print()
  with tqdm.notebook.tqdm(total=100, bar_format=TQDM_BAR_FORMAT) as pbar:
    with io.capture_output() as captured:
      os.chdir("/content")
      os.system("git clone --recurse-submodules https://github.com/reedacartwright/mcberepair.git")
      os.chdir("mcberepair")
      pbar.update(40)
      os.system("cmake -S . -B build -DCMAKE_BUILD_TYPE=Release")
      os.system("cmake --build build --parallel")
      pbar.update(40)
      os.chdir("/content")
      os.system("mkdir test")
      os.chdir("test")
      os.system("ln -sf /content/mcberepair/build/mcberepair .")
      pbar.update(20)
  print("初期設定完了.")

except subprocess.CalledProcessError:
  print(captured)
  raise

#main
fname = "tmp.zip"
print()
print()
print('\033[1m'+"---------- [機能一覧] ----------"+'\033[0m')

def timestampJST(forLog=False,isJP=True):
  if isJP:
    t_delta = datetime.timedelta(hours=9)
    tz = datetime.timezone(t_delta, 'JST')
    tzname = "JST"
  else:
    t_delta = datetime.timedelta(hours=0)
    tz = datetime.timezone(t_delta, 'UTC')
    tzname = "UTC"
  
  dt = datetime.datetime.now(tz)
  if forLog:
    return str(dt.year)+"-"+str(dt.month).zfill(2)+"-"+str(dt.day).zfill(2)+" "+str(dt.hour).zfill(2)+":"+str(dt.minute).zfill(2)+":"+str(dt.second).zfill(2)+tzname
  else:
    timestamp = str(dt.year)+str(dt.month).zfill(2)+str(dt.day).zfill(2)+str(dt.hour).zfill(2)+str(dt.minute).zfill(2)+str(dt.second).zfill(2)+tzname
    logtimestamp = str(dt.year)+"-"+str(dt.month).zfill(2)+"-"+str(dt.day).zfill(2)+" "+str(dt.hour).zfill(2)+":"+str(dt.minute).zfill(2)+":"+str(dt.second).zfill(2)+tzname
    return timestamp,logtimestamp

def upload_zip(b):
  global fname
  os.system("rm -rf "+fname.replace(".zip","*"))
  uploaded = files.upload()
  try:
    with io.capture_output() as captured:
      
      fname = list(uploaded.keys())[0]
      os.system("rm -rf ./*.mcberepair.*UTC.mcworld")
      os.system("rm -rf ./*.mcberepair.*UTC.zip")
      os.system("rm -rf ./*.mcberepair.*JST.mcworld")
      os.system("rm -rf ./*.mcberepair.*JST.zip")
      os.system("rm -rf ./*.org")
      os.system("rm -rf "+fname.replace(".zip",""))
      os.system("unzip "+fname)
      os.system("cp -r "+fname.replace(".zip","")+" "+fname.replace(".zip",".org"))
      os.system("rm -rf "+fname)
    print(timestampJST(forLog=True)+" [アップロード] "+'\033[32m'+os.path.basename(fname.replace(".zip",""))+'\033[0m'+" がアップロードされ作業対象になりました.")

  except:
    pass

def delete_pticks(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("./mcberepair listkeys "+fname.replace(".zip","")+" > list.tsv")
        os.system("awk '$6 == 51 && $2 > 4096 {print $1}' list.tsv > pending_ticks.tsv")
        os.system("./mcberepair rmkeys "+fname.replace(".zip","")+" < pending_ticks.tsv")
        nkeys = np.loadtxt("pending_ticks.tsv",delimiter=" ",dtype="|S").shape[0]
        os.system("rm -rf *.tsv")
      print(timestampJST(forLog=True)+" [pending tickの消去] "+str(nkeys)+"個のkeyが該当し削除されました.")
    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[警告]"+"\033[0m"+" ワールドデータ(.zip)がアップロードされていません.")

def delete_remote_players(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("./mcberepair listkeys "+fname.replace(".zip","")+" > list.tsv")
        os.system("awk '$0 ~ /player_/ {print $1}' list.tsv > remote_players.tsv")
        os.system("./mcberepair rmkeys "+fname.replace(".zip","")+" < remote_players.tsv")
        nkeys = np.loadtxt("remote_players.tsv",delimiter=" ",dtype="|S").shape[0]
        os.system("rm -rf *.tsv")
      print(timestampJST(forLog=True)+" [リモートプレーヤーデータの削除] "+str(nkeys)+"個のkeyが該当し削除されました.")

    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[警告]"+"\033[0m"+" ワールドデータ(.zip)がアップロードされていません.")

def reset_proc(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        os.system("rm -rf "+fname.replace(".zip",""))
        os.system("cp -r "+fname.replace(".zip",".org")+" "+fname.replace(".zip",""))
      print(timestampJST(forLog=True)+" [リセット] 全ての作業がリセットされました.")

    except subprocess.CalledProcessError:
      print(captured)
      raise
  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[警告]"+"\033[0m"+" ワールドデータ(.zip)がアップロードされていません.")

def download_zip(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        
        os.system("rm -rf ./*.mcberepair.*UTC.zip")
        os.system("rm -rf ./*.mcberepair.*JST.zip")
        timestamp,logtimestamp = timestampJST()
        os.chdir(fname.replace(".zip",""))
        outputfname = os.path.basename( fname.replace(".zip",".mcberepair."+timestamp+".zip") )
        os.system("zip "+outputfname+" *")
        os.system("zip -r "+outputfname+" *")
        os.system("mv "+outputfname+" ../")
        os.chdir("../")

      files.download(outputfname)
      print(logtimestamp+" [ダウンロード] "+outputfname+" をダウンロードしました.")

    except subprocess.CalledProcessError:
      print(captured)
      raise

  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[警告]"+"\033[0m"+" ワールドデータ(.zip)がアップロードされていません.")

def download_mcworld(b):
  global fname
  if os.path.exists(fname.replace(".zip","")):
    try:
      with io.capture_output() as captured:
        
        os.system("rm -rf ./*.mcberepair.*UTC.mcworld")
        os.system("rm -rf ./*.mcberepair.*JST.mcworld")
        timestamp,logtimestamp = timestampJST()
        os.chdir(fname.replace(".zip",""))
        outputfname = os.path.basename( fname.replace(".zip",".mcberepair."+timestamp+".zip") )
        os.system("zip "+outputfname+" *")
        os.system("zip -r "+outputfname+" *")
        os.system("mv "+outputfname+" ../")
        os.chdir("../")
        os.system("mv "+outputfname+" "+outputfname.replace(".zip",".mcworld"))

      files.download(outputfname.replace(".zip",".mcworld"))
      print(logtimestamp+" [ダウンロード] "+outputfname.replace(".zip",".mcworld")+" をダウンロードしました.")

    except subprocess.CalledProcessError:
      print(captured)
      raise

  else:
    print(timestampJST(forLog=True)+" "+"\033[33m"+"[警告]"+"\033[0m"+" ワールドデータ(.zip)がアップロードされていません.")
  

button0 = widgets.Button(description="Delete pticks")
button1 = widgets.Button(description="Delete remote players")
button2 = widgets.Button(description="Reset")
button3 = widgets.Button(description="Download as zip")
button4 = widgets.Button(description="Download as mcworld")
button5 = widgets.Button(description="Upload zip file")
button0.on_click(delete_pticks)
button1.on_click(delete_remote_players)
button2.on_click(reset_proc)
button3.on_click(download_zip)
button4.on_click(download_mcworld)
button5.on_click(upload_zip)
display(button5)
print("  ワールドをアップロードします "+'\033[1m'+"(.zip ファイルをアップしてください.)"+'\033[0m')
print("   *事前にワールドフォルダ(「TbLUY1uSXQA=」みたいなやつ)をzipに圧縮してください.")
print("   *"+'\033[1m'+"アップロードファイルのセレクタは下の方([ログ]の下)に出ます."+'\033[0m')
print()
display(button0)
print("  4096バイトを超えるpending tickデータを一括消去します.")
display(button1)
print("  リモートプレーヤーデータを一括消去します.")
display(button2)
print("  作業をすべてやり直し、ワールドをアップロード時の状態に戻します.")
print()
display(button3)
print("  作業後のワールドをzipファイルとしてダウンロードします.")
display(button4)
print("  作業後のワールドをmcworldファイルとしてダウンロードします.")
print()
print('\033[1m'+"---------- [ログ] ----------"+'\033[0m')